In [1]:
!pip install -U -q PyDrive

    100% |████████████████████████████████| 993kB 18.9MB/s 


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1H8Sj8FvpLwz1Ti_4ZqthWWtz1oaga3Fu'}) 
downloaded.GetContentFile('carvan_train.csv')

In [0]:
downloaded = drive.CreateFile({'id':'1xKGUN3FfiLG3uBtpwqdQC-NVKChGmWqi'}) 
downloaded.GetContentFile('carvan_test.csv')

In [0]:
#Basic Libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from sklearn.model_selection import train_test_split,KFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import datetime
import random
import tensorflow as tf
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#Classification Libraries
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#Other Data Processing libraries
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

In [0]:
car_train = pd.read_csv('carvan_train.csv')
car_test = pd.read_csv('carvan_test.csv')

In [8]:
car_train.shape

(5822, 86)

In [9]:
car_test.shape

(4000, 85)

In [10]:
car_train.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86
count,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,...,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000
mean,24.253349,1.110615,2.678805,2.991240,5.773617,0.696496,4.626932,1.069907,3.258502,6.183442,...,0.005325,0.006527,0.004638,0.570079,0.000515,0.006012,0.031776,0.007901,0.014256,0.059773
std,12.846706,0.405842,0.789835,0.814589,2.856760,1.003234,1.715843,1.017503,1.597647,1.909482,...,0.072782,0.080532,0.077403,0.562058,0.022696,0.081632,0.210986,0.090463,0.119996,0.237087
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,1.000000,2.000000,2.000000,3.000000,0.000000,4.000000,0.000000,2.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,30.000000,1.000000,3.000000,3.000000,7.000000,0.000000,5.000000,1.000000,3.000000,6.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,35.000000,1.000000,3.000000,3.000000,8.000000,1.000000,6.000000,2.000000,4.000000,7.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,41.000000,10.000000,5.000000,6.000000,10.000000,9.000000,9.000000,5.000000,9.000000,9.000000,...,1.000000,1.000000,2.000000,7.000000,1.000000,2.000000,3.000000,2.000000,2.000000,1.000000


In [11]:
 car_train['V42'].value_counts()

3    1932
4    1854
5     733
2     651
6     355
7     131
8      70
1      49
0      25
9      22
Name: V42, dtype: int64

In [13]:
 car_train['V43'].value_counts()

3    1524
4     902
6     901
1     587
5     583
7     474
8     426
2     425
Name: V43, dtype: int64

In [14]:
car_train['V45'].value_counts()

0    5740
2      30
3      23
4      17
1       7
6       4
5       1
Name: V45, dtype: int64

In [15]:
car_train['V59'].value_counts()

0    2666
4    1226
3     920
2     535
1     161
6     155
5     149
7       9
8       1
Name: V59, dtype: int64

In [16]:
car_train['V31'].value_counts()

9    949
0    760
7    724
6    604
1    530
5    520
4    499
8    428
2    426
3    382
Name: V31, dtype: int64

In [0]:
# Listing down categorical columns
cat_cols = ['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27'
            ,'V28','V29','V30','V32','V33','V34','V35','V36','V37','V38','V39','V40','V41']

In [0]:
for col in cat_cols:
  car_train[col] = car_train[col].astype('category')

In [0]:
for col in cat_cols:
  car_test[col] = car_test[col].astype('category')

In [0]:
car_train['V86'] = car_train['V86'].astype('category')

In [22]:
car_train.describe()

,V31,V42,V43,V44,V45,V46,V47,V48,V49,V50,...,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85
count,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,...,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000,5822.000000
mean,4.771728,3.784438,4.236345,0.771213,0.040021,0.071625,2.970457,0.048265,0.175369,0.009447,...,0.076606,0.005325,0.006527,0.004638,0.570079,0.000515,0.006012,0.031776,0.007901,0.014256
std,3.089837,1.317783,2.007150,0.958623,0.362680,0.499980,2.920669,0.531346,0.897222,0.244675,...,0.377569,0.072782,0.080532,0.077403,0.562058,0.022696,0.081632,0.210986,0.090463,0.119996
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,4.000000,4.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.000000,4.000000,6.000000,2.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.000000,9.000000,8.000000,3.000000,6.000000,4.000000,8.000000,7.000000,7.000000,9.000000,...,8.000000,1.000000,1.000000,2.000000,7.000000,1.000000,2.000000,3.000000,2.000000,2.000000


In [23]:
car_train.isnull().sum().sum()

0

In [24]:
car_test.isnull().sum().sum()

0

In [25]:
car_train['V86'].value_counts()

0    5474
1     348
Name: V86, dtype: int64

In [26]:
car_train['V86'].value_counts()[0]/len(car_train) * 100

94.02267262109241

In [27]:
car_train['V86'].value_counts()[1]/len(car_train) * 100

5.977327378907591

In [0]:
X = car_train.drop('V86', axis=1)
Y = car_train['V86']

In [30]:
#Our data has great imbalance, 
#i.e, there are very less records for fraud class(Target:1). hence using this for sampling from each group

from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(X, Y):
    print("Train:", train_index, "Test:", test_index)
    actual_Xtrain, actual_Xtest = X.iloc[train_index], X.iloc[test_index]
    actual_Ytrain, actual_Ytest = Y.iloc[train_index], Y.iloc[test_index]

actual_Xtrain = actual_Xtrain.values
actual_Xtest = actual_Xtest.values
actual_Ytrain = actual_Ytrain.values
actual_Ytest = actual_Ytest.values


train_unique_label, train_counts_label = np.unique(actual_Ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(actual_Ytest, return_counts=True)
print('---' * 85)

print('Label Distributions: \n')
print(train_counts_label/ len(actual_Ytrain))
print(test_counts_label/ len(actual_Ytest))

Train: [ 198  676  330 ... 5077 1170 5819] Test: [4487 3556 4140 ...  324 5288 1227]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Label Distributions: 

[0.94030492 0.05969508]
[0.93991416 0.06008584]


In [0]:
import time
t0 = time.time()
logr=LogisticRegression(penalty="l1",class_weight="balanced",random_state=2)
logr.fit(actual_Xtrain, actual_Ytrain)
t1 = time.time()
print("Fitting Logistic Regression Model took {} sec".format(t1 - t0))

Fitting Logistic Regression Model took 96.01282811164856 sec


In [0]:
#predicting on our validaton data
pred_test = logr.predict(actual_Xtest)

In [0]:
#checking for AUC Score metric
roc_auc_score(pred_test,actual_Ytest)

0.5449620427881298

In [0]:
from sklearn.metrics import fbeta_score

In [0]:
car_test.shape

(5822, 154)

In [0]:
car_train.shape

(5822, 154)

In [0]:
pred_all = logr.fit(X,Y)

In [0]:
car_pred = pred_all.predict(X_test)

In [0]:
fbeta_score(car_pred, Y_test, beta=2.0)

0.1646937725167267

In [0]:
car_pred = logr.predict(X_test)

In [0]:
fbeta_score(car_pred, Y_test,average='weighted', beta=2.0)

0.6654595947486318

In [0]:
predictions = pd.DataFrame(list(zip(car_pred)),columns=["V86"])

In [0]:
predictions.to_csv("Manoj_Garikapati_Project2_Part2.csv")

In [0]:
file = drive.CreateFile({'parents':[{u'id': '11GVu6JUMmRxjJsneR5cJiFng-XNFVrLN'}]}) 
file.SetContentFile("Manoj_Garikapati_Project2_Part2.csv")
file.Upload()

In [0]:
fbeta_score(car_pred, Y_test,beta=2.0)

0.16046867040244522

In [0]:
#Random Forests using randomized search wiht below parameters
from scipy.stats import randint as sp_randint
param_dist_rf = {"n_estimators":[10,100,500,700],
              "max_depth":[10,20,30,40,50,60,70],
              "max_features": sp_randint(30, 60),
              "min_samples_split": sp_randint(5, 11),
              "min_samples_leaf": sp_randint(5, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"],
              'class_weight':['balanced']}

In [58]:
for train_index, test_index in sss.split(X, Y):
    print("Train:", train_index, "Test:", test_index)
    actual_Xtrain, actual_Xtest = X.iloc[train_index], X.iloc[test_index]
    actual_Ytrain, actual_Ytest = Y.iloc[train_index], Y.iloc[test_index]

Train: [ 198  676  330 ... 5077 1170 5819] Test: [4487 3556 4140 ...  324 5288 1227]


In [59]:
#Fitting on validation train data
from sklearn.metrics import precision_score, cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,KFold, RandomizedSearchCV
n_iter_search = 20
rf_clf = RandomForestClassifier(verbose=1,n_jobs=-1)
random_search = RandomizedSearchCV(rf_clf, param_distributions=param_dist_rf,
                                   n_iter=n_iter_search,scoring='roc_auc')
random_search.fit(actual_Xtrain,actual_Ytrain)
#This takes quite some time to execute and increases with number of iterations

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  1

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=20, n_jobs=None,
          param_distributions={'n_estimators': [10, 100, 500, 700], 'max_depth': [10, 20, 30, 40, 50, 60, 70], 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f33dabfd2b0>, 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f33dabfd400>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f33dabfd550>, 'bootstrap': [True, False], 'criterion': ['

In [60]:
#predicting on our validaton data
pred_test = random_search.predict(actual_Xtest)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 700 out of 700 | elapsed:    0.3s finished


In [61]:
random_search.best_params_

{'bootstrap': True,
 'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 40,
 'max_features': 44,
 'min_samples_leaf': 10,
 'min_samples_split': 10,
 'n_estimators': 700}

In [62]:
roc_auc_score(pred_test,actual_Ytest)

0.5804966912995339

In [63]:
fbeta_score(pred_test, actual_Ytest, beta=2.0)

0.211864406779661

In [64]:
random_search.best_params_

{'bootstrap': True,
 'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 40,
 'max_features': 44,
 'min_samples_leaf': 10,
 'min_samples_split': 10,
 'n_estimators': 700}

In [0]:
rand_best = random_search.best_estimator_

In [0]:
rand_best.fit(X,Y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=60, max_features=59, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=6, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [0]:
car_pred = rand_best.predict(car_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished


In [31]:
# Using Naive Bayes for classification

clf_NB = BernoulliNB()
clf_NB.fit(actual_Xtrain,actual_Ytrain)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [0]:
pred_Ytest = clf_NB.predict(actual_Xtest)

In [33]:
from sklearn.metrics import fbeta_score
fbeta_score(actual_Ytest,pred_Ytest, beta=2)

0.23809523809523814

In [34]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(actual_Ytest,pred_Ytest))

[[1035   60]
 [  53   17]]


In [35]:
from sklearn.neural_network import *
MLPClassifier(activation='relu', alpha=1e-05,
       batch_size='auto', beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(64), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       tol=0.001, validation_fraction=0.1, verbose=True,
       warm_start=False)
clf_MLP = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(64))

clf_MLP.fit(actual_Xtrain, actual_Ytrain)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=64, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
y_pred_MLP = clf_MLP.predict(actual_Xtest)

In [37]:
fbeta_score(actual_Ytest,y_pred_MLP,beta=2.0)

0.15527950310559008

In [38]:
confusion_matrix(actual_Ytest,y_pred_MLP)

array([[1063,   32],
       [  60,   10]])

In [39]:
# Using Naive Bayes for classification

clf_NB = GaussianNB()
clf_NB.fit(actual_Xtrain,actual_Ytrain)

GaussianNB(priors=None, var_smoothing=1e-09)

In [40]:
y_pred_GNB = clf_NB.predict(actual_Xtest)
fbeta_score(actual_Ytest,y_pred_GNB,beta=2.0)

0.2531152647975078

In [41]:
confusion_matrix(actual_Ytest,y_pred_GNB)

array([[156, 939],
       [  5,  65]])

In [42]:
clf_NB.fit(X,Y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
original_car_pred = clf_NB.predict(car_test)

In [0]:
predictions = pd.DataFrame(list(zip(original_car_pred)),columns=["V86"])

In [0]:
predictions.to_csv("Manoj_Garikapati_Project2_Part2.csv")
file = drive.CreateFile({'parents':[{u'id': '11GVu6JUMmRxjJsneR5cJiFng-XNFVrLN'}]}) 
file.SetContentFile("Manoj_Garikapati_Project2_Part2.csv")
file.Upload()

In [46]:
predictions.shape

(4000, 1)

In [0]:
from sklearn.naive_bayes import MultinomialNB
clf_MB = MultinomialNB()

In [49]:
clf_MB.fit(actual_Xtrain,actual_Ytrain)
y_pred_MB = clf_MB.predict(actual_Xtest)
fbeta_score(actual_Ytest,y_pred_MB,beta=2.0)

0.38599640933572715

In [50]:
clf_MB.fit(X,Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
original_car_pred = clf_NB.predict(car_test)

In [0]:
predictions = pd.DataFrame(list(zip(original_car_pred)),columns=["V86"])

In [0]:
predictions.to_csv("Manoj_Garikapati_Project2_Part2.csv")
file = drive.CreateFile({'parents':[{u'id': '11GVu6JUMmRxjJsneR5cJiFng-XNFVrLN'}]}) 
file.SetContentFile("Manoj_Garikapati_Project2_Part2.csv")
file.Upload()